In [88]:
import numpy as np
import scipy.stats

In [2]:
import tensorflow as tf

In [4]:
M = 5
N = 2
T = 3
h = 2
s = 3

In [7]:
A_np = np.random.randn(M, h)
C_np = np.random.randn(s, T)
B_np = np.random.randn(h, N, s)

In [11]:
print(A_np)
print(B_np)
np.tensordot(A_np, B_np, axes=1)

[[-0.71722232 -1.56709009]
 [ 0.41213077  1.11844308]
 [-0.388832    1.03975487]
 [ 0.3505914   1.10913787]
 [-0.99167438  0.08882283]]
[[[-2.77781815 -0.73546259 -0.63187946]
  [ 0.41161213  0.68682337 -0.59435527]]

 [[-0.83167171 -0.06335832 -1.07703616]
  [-1.11668467 -0.66917665  0.69701998]]]


array([[[ 3.29561768,  0.62677838,  2.14101074],
        [ 1.45472807,  0.55605504, -0.66600824]],

       [[-2.0750018 , -0.37396944, -1.4650206 ],
        [-1.07931021, -0.46537495,  0.53462508]],

       [[ 0.21536987,  0.22009427, -0.87415863],
        [-1.32112628, -0.96283857,  0.95583426]],

       [[-1.89631774, -0.32811997, -1.41611309],
        [-1.09424959, -0.5014148 ,  0.56471542]],

       [[ 2.68081966,  0.72371174,  0.53095327],
        [-0.50737229, -0.7405433 ,  0.65131818]]])

In [16]:
A_tf = tf.Variable(A_np)
C_tf = tf.Variable(C_np)
B_tf = tf.Variable(B_np)

In [26]:
AB_tf = tf.tensordot(A_tf, B_tf,axes = [[1], [0]])
sess = tf.Session()
sess.run(tf.global_variables_initializer())
AB = sess.run(AB_tf)

In [27]:
print(AB)

[[[ 3.29561768  0.62677838  2.14101074]
  [ 1.45472807  0.55605504 -0.66600824]]

 [[-2.0750018  -0.37396944 -1.4650206 ]
  [-1.07931021 -0.46537495  0.53462508]]

 [[ 0.21536987  0.22009427 -0.87415863]
  [-1.32112628 -0.96283857  0.95583426]]

 [[-1.89631774 -0.32811997 -1.41611309]
  [-1.09424959 -0.5014148   0.56471542]]

 [[ 2.68081966  0.72371174  0.53095327]
  [-0.50737229 -0.7405433   0.65131818]]]


In [30]:
AB_tf_C_tf = tf.tensordot(AB_tf, C_tf, axes=[[2], [1]])
ABC = sess.run(AB_tf_C_tf)

In [31]:
print(ABC)

[[[ 0.75527036 -5.02275331  0.28318609]
  [-0.94931532  1.75120339 -0.10835656]]

 [[-0.56839122  3.45147461 -0.19421431]
  [ 0.7016425  -1.50424785 -0.0387348 ]]

 [[-0.75552538  2.17709947 -0.11943444]
  [ 0.83885192 -3.36712311 -0.93422276]]

 [[-0.58075925  3.34510851 -0.18799793]
  [ 0.70984948 -1.63992902 -0.10602635]]

 [[-0.34653352 -1.09476287  0.06567728]
  [ 0.30330604 -2.73194579 -1.19500948]]]


In [32]:
def f(theta, x, returngrad=False):
    fval = (theta[0])*(theta[1] - x)
    if returngrad==False:
        return fval
    else:
        if (np.isscalar(x)):
            xshape = ()
        else:
            xshape = x.shape
        dfdtheta = np.zeros(theta.shape + xshape)
        # want this to broadcast correctly regardless of whether x is a matrix or vector
        dfdtheta[0,...] = fval/theta[0]
        dfdtheta[1,...] = theta[0]
        dfdtheta[2,...] = 0
        return fval, dfdtheta
    
def g(theta, x, returngrad=False):
    if (np.isscalar(x)):
        xshape = ()
        gval = theta[2]
    else:
        xshape = x.shape
        gval = np.ones(xshape)*theta[2]
        
    if returngrad==False:
        return gval
    else:
        dgdtheta = np.zeros(theta.shape + xshape)
        # want this to broadcast correctly regardless of whether x is a matrix or vector
        dgdtheta[0,...] = 0
        dgdtheta[1,...] = 0
        dgdtheta[2,...] = 1
        return gval, dgdtheta

In [34]:
theta = 0.5
omtheta = 1.-theta
alpha = np.zeros(2)
alpha[0] = 0.5/(theta*omtheta)
alpha[1] = alpha[0]*(omtheta*omtheta + theta*theta)

In [91]:
M = 10
k = 0.5
grid = np.arange(-M,(M+1))*k
yim1 = grid.copy()
h = 0.5

In [57]:
# play with sparse tensors a bit
N = 2*M+1
ind0 = np.arange(0,N)
ind = np.repeat(np.array([ind0]).T,3,axis=1)
val = np.repeat(1,N)
ds = [N,N,N]

In [64]:
test = tf.SparseTensor(indices=ind,values=val,dense_shape=ds)

In [65]:
testsparse = sess.run(testd)

In [100]:
col1 = np.ones((N,1))
colr = np.array([np.arange(0,N)]).T

In [107]:
# create the sparse A tensor
ds = [N,N,N]
val = []
ind = np.zeros((0,3))
tol = N
th = np.array([1.,0.5,0.75])
col1 = np.ones((N,1))
colr = np.array([np.arange(0,N)]).T
for i in range(N):
    for j in range(N):
        if (abs(i-j) > tol):
            continue
            
        yi = grid[i]
        ystar = grid[j]
        mv = ystar + (alpha[0]*f(th,ystar) - alpha[1]*f(th,yim1))*omtheta*h
        g2star = np.square(g(th,ystar))
        g2m1 = np.square(g(th,yim1))
        vv = np.maximum(0, alpha[0]*g2star - alpha[1]*g2m1)*omtheta*h
        thesevals = scipy.stats.norm.pdf(x=yi, loc=mv, scale=np.sqrt(vv))
        indblock = np.hstack([i*col1,j*col1,colr])
        ind = np.vstack([ind, indblock])
        val.append(thesevals.tolist())

In [115]:
val = np.array(val).flatten()

In [116]:
test = tf.SparseTensor(indices=ind,values=val,dense_shape=ds)

In [117]:
testsparse = sess.run(test)